In [1]:
import numpy as np
import bilby
import pylab as plt
from gwsnr import GWSNR as snr

# bilby snr with injection

In [45]:
injection_parameters = dict(mass_1=10.,
                            mass_2=10.,
                            luminosity_distance=100.0,
                            theta_jn=0.5,
                            psi=0.5,
                            phase=0.5,
                            geocent_time=1246527224.169434,
                            ra=0.0,
                            dec=0.0,
                            a_1=0.5,
                            a_2=0.5,
                            tilt_1=0.5,
                            tilt_2=0.5,
                            phi_12=0.5,
                            phi_jl=0.5,
                        )

In [46]:
def duration_fn(mass_1, mass_2, f_min):
    safety = 1.2
    approx_duration = safety * snr_.findchirp_chirptime(mass_1, mass_2, f_min)
    return np.ceil(approx_duration + 4.0)

In [47]:
sampling_frequency = 4096.
minimum_frequency = 20.
duration = duration_fn(mass_1=injection_parameters['mass_1'],
                       mass_2=injection_parameters['mass_2'],
                       f_min=20.)

waveform_arguments = dict(waveform_approximant = 'IMRPhenomXPHM',
                            reference_frequency = 30.,
                            minimum_frequency = minimum_frequency) 
waveform_generator = bilby.gw.WaveformGenerator(duration = duration,
    sampling_frequency = sampling_frequency,
    frequency_domain_source_model = bilby.gw.source.lal_binary_black_hole,
    parameters=injection_parameters,
    waveform_arguments = waveform_arguments
    )

In [48]:
asd_file = ['aLIGO_O4_high_asd.txt', 'aLIGO_O4_high_asd.txt', 'AdV_asd.txt']
ifos = bilby.gw.detector.InterferometerList(['H1', 'L1', 'V1'])

for i in range(3):
    ifos[i].power_spectral_density = bilby.gw.detector.PowerSpectralDensity(asd_file=asd_file[i])

ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency,
    duration=duration,
    start_time=injection_parameters["geocent_time"] + 2 - duration,
)

In [49]:
# make sure you getting right psd
print(ifos[0].power_spectral_density)

PowerSpectralDensity(psd_file='None', asd_file='/Users/hemantaph/anaconda3/envs/lal2/lib/python3.10/site-packages/bilby/gw/detector/noise_curves/aLIGO_O4_high_asd.txt')


In [50]:
bilby.core.utils.logger.disabled = True
ifos.inject_signal(
    waveform_generator=waveform_generator, parameters=injection_parameters
);

In [51]:
# from bilby injection
print(f"snr_L1 = {ifos[1].meta_data['optimal_SNR']}")
print(f"snr_H1 = {ifos[0].meta_data['optimal_SNR']}")
print(f"snr_V1 = {ifos[2].meta_data['optimal_SNR']}")

snr_L1 = 117.6773296648426
snr_H1 = 74.76707718037684
snr_V1 = 35.17614663474223


# inner product

In [2]:
# nsamples is use for fitting. default: nsamples=100
# nsamples with higher value will give better result.
# for BNS events nsamples can be kept low
# with defult bilby's psd
snr_ = snr(waveform_approximant = 'IMRPhenomXPHM', snr_type = 'inner_product')

Given: IMR waveform, {self.waveform_approximant}.
psds not given. Choosing bilby's default psds
given psds:  {'L1': 'aLIGO_O4_high_asd.txt', 'H1': 'aLIGO_O4_high_asd.txt', 'V1': 'AdV_asd.txt'}


In [3]:
# with inner_product with gwsnr package
# hp_inner_hc included in the package
snr_test = snr_.snr(mass_1=10.,
                    mass_2=10.,
                    luminosity_distance=100.0,
                    iota=0.5,
                    psi=0.5,
                    phase=0.5,
                    geocent_time=1246527224.169434,
                    ra=0.0,
                    dec=0.0,
                    a_1=0.5,
                    a_2=0.5,
                    tilt_1=0.5,
                    tilt_2=0.5,
                    phi_12=0.5,
                    phi_jl=0.5,
)
print(f"snr_L1 = {snr_test['L1']}")
print(f"snr_H1 = {snr_test['H1']}")
print(f"snr_V1 = {snr_test['V1']}")

solving SNR with inner product


  0%|                                                                         | 0/1 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 20.24it/s]

snr_L1 = [117.59034621]
snr_H1 = [74.77707807]
snr_V1 = [35.24596307]


In [3]:
# with inner_product with gwsnr package
# hp_inner_hc included in the package
# note: I have made some changes in the code
# in the function noise_weighted_inner_prod of GWSNR class
snr_test = snr_.snr(mass_1=10.,
                    mass_2=10.,
                    luminosity_distance=100.0,
                    iota=0.5,
                    psi=0.5,
                    phase=0.5,
                    geocent_time=1246527224.169434,
                    ra=0.0,
                    dec=0.0,
                    a_1=0.5,
                    a_2=0.5,
                    tilt_1=0.5,
                    tilt_2=0.5,
                    phi_12=0.5,
                    phi_jl=0.5,
)
print(f"snr_L1 = {snr_test['L1']}")
print(f"snr_H1 = {snr_test['H1']}")
print(f"snr_V1 = {snr_test['V1']}")

solving SNR with inner product


100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.81it/s]

snr_L1 = [120.33915932]
snr_H1 = [74.85566925]
snr_V1 = [41.25122545]


In [6]:
type(snr_.power_spectral_density_pycbc(psd='aLIGOaLIGODesignSensitivityT1800044'))

bilby.gw.detector.psd.PowerSpectralDensity